In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
if (False):
    import tarfile
    tar = tarfile.open("car_ims.gz")
    tar.extractall()
    tar.close()

# Imports

In [ ]:
import time
from tqdm import tqdm
import gc
import random
import os
from shutil import move
from pathlib import Path
from matplotlib import pyplot as plt

In [ ]:
gc.collect()

In [ ]:
if (False):
    dataDir = 'data/irlFort/'
    imageNames = open('red-cars.txt').readlines()
    imageNames = [x.strip() for x in imageNames]
    #print(len(imageNames))
    #print(imageNames)
    
    for imageName in imageNames:
        move(dataDir+'car_ims/'+imageName, dataDir+'trainA/'+imageName)

In [ ]:
if (False):
    dataDir = 'data/irlFort/'
    imageNames = open('blue-cars.txt').readlines()
    imageNames = [x.strip() for x in imageNames]
    #print(len(imageNames))
    #print(imageNames)
    
    for imageName in imageNames:
        try:
            move(dataDir+'car_ims/'+imageName, dataDir+'trainB/'+imageName)
        except:
            pass

In [ ]:
if (False):
    dataDir = 'data/irlFort/'
    imageNames = os.listdir(dataDir+'trainA')
    random.shuffle(imageNames)
    for imageName in imageNames[0:2000]:
        os.remove(dataDir+'trainA/'+imageName)

In [ ]:
if (False):
    
    data_dir = Path('data/irlFort/')
    
    imagesTrainA = list((data_dir/'trainA').iterdir())
    imagesTrainB = list((data_dir/'trainB').iterdir())

    random.shuffle(imagesTrainA)
    random.shuffle(imagesTrainB)

    for img in imagesTrainA[0:int(len(imagesTrainA)*0.2)]:
        os.rename(img, data_dir/'testA'/img.name)

    for img in imagesTrainB[0:int(len(imagesTrainB)*0.2)]:
        os.rename(img, data_dir/'testB'/img.name)

# Cycle GAN

In [ ]:
from cgan.options.train_options import *
from cgan.options.test_options import *

In [ ]:

# ---------- Training ----------
#opt = TrainOptions().parse(['--dataroot', 'data/irlFort', '--nThreads', '8', '--no_dropout',
#                           '--niter', '30', '--niter_decay', '15', '--name', 'nodrop', '--gpu_ids', '0',
#                            '--batchSize', '1'])

#opt = TrainOptions().parse(['--dataroot', 'data/irlFort', '--nThreads', '8', '--no_dropout',
#                           '--niter', '30', '--niter_decay', '0', '--name', 'nodrop', '--gpu_ids', '0',
#                            '--batchSize', '1', '--continue_train', '--epoch_count', '30', '--which_epoch', '15'])


# ---------- Testing ----------
opt = TestOptions().parse(['--dataroot', 'data/red_car_imgs_01', '--nThreads', '8', '--no_dropout',
                           '--which_direction', 'AtoB', '--name', 'nodrop', '--gpu_ids', '0',
                            '--batchSize', '1', '--which_epoch', 'latest', '--serial_batches'])

In [ ]:
from cgan.options.train_options import TrainOptions
from cgan.data.data_loader import CreateDataLoader
from cgan.models.models import create_model

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
dataset_size

In [ ]:
for data in dataset:
    print(data['A'].shape)
    break

In [ ]:
model = create_model(opt)

In [ ]:
def show_img(im, ax=None, figsize=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return ax

In [ ]:
outputNum = 1
def show_grid(ims):
    fig,axes = plt.subplots(2,3,figsize=(9,6))
    for i,ax in enumerate(axes.flat): show_img(ims[i], ax);
    fig.tight_layout()
    
    global outputNum
    plt.savefig('outputs/output' + str(outputNum) + '.png')
    outputNum += 1

In [ ]:
def get_one(data):
    model.set_input(data)
    model.test()
    return list(model.get_current_visuals().values())

In [ ]:
def drawImgs():
    test_ims = []
    for i,o in enumerate(dataset):
        if i>1:
            break
        test_ims.append(get_one(o))
    for i in range(1):
        show_grid(test_ims[i])

In [ ]:
# --- Testing ---

for i, data in enumerate(dataset):
    model.set_input(data)
    model.test()
    myValues = list(model.get_current_visuals().values())
    img_path = model.get_image_paths()
    print('%04d: process image... %s' % (i, img_path))
    
    # 4th value for BtoA, 1st value for AtoB
    plt.imshow(myValues[1])
    plt.xticks([])
    plt.yticks([])
    
    fName = Path(img_path[0]).name
    plt.savefig("results/" + fName)
    
    #show_grid(myValues)
    #if (i == 1):
        #break

In [ ]:
if (False):

    # --- Training ---
    total_steps = 0

    for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
        epoch_start_time = time.time()
        iter_data_time = time.time()
        epoch_iter = 0

        for i, data in tqdm(enumerate(dataset)):
            iter_start_time = time.time()
            if total_steps % opt.print_freq == 0: t_data = iter_start_time - iter_data_time
            total_steps += opt.batchSize
            epoch_iter += opt.batchSize
            model.set_input(data)
            model.optimize_parameters()

            if total_steps % opt.display_freq == 0:
                save_result = total_steps % opt.update_html_freq == 0
                drawImgs()

            if total_steps % opt.print_freq == 0:
                errors = model.get_current_errors()
                t = (time.time() - iter_start_time) / opt.batchSize

            if total_steps % opt.save_latest_freq == 0:
                print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
                model.save('latest')

            iter_data_time = time.time()
        if epoch % opt.save_epoch_freq == 0:
            print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))
            model.save('latest')
            model.save(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' %
              (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
        model.update_learning_rate()
        drawImgs()